Getting the access token

In [11]:
import requests

client_id = "client_id"
client_secret = 'client_secret'

auth_url = 'https://accounts.spotify.com/api/token'
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

auth_response = requests.post(auth_url, data=data)
access_token = auth_response.json().get('access_token')
access_token

'BQCRz4UXGEMFNWoCDBKb0qEf3xRrqQ2VhAJk8TP4-JWveqv8l-yBa6I04RD9aIlcBQARqkkYKzJtHV_i9PnMxcSig3KTMabYvJjUe-sQpdrmyvHtb63U'

In [12]:
import csv
import requests

# Set up your access token
access_token = access_token

# Set up the API endpoint
endpoint = "https://api.spotify.com/v1/search"

# Set up the query parameters
query_params = {
    "q": "artist:'One Direction'",
    "type": "track",
    "limit": 50  # Adjust the limit as per your needs
}

# Set up the request headers with the access token
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Send the initial GET request to the Spotify API
response = requests.get(endpoint, params=query_params, headers=headers)

# Parse the response JSON
response_json = response.json()
print(response_json)
# Extract the relevant data from the response
tracks = response_json["tracks"]["items"]

# Open the CSV file in write mode
with open("songs.csv", mode="w", newline="") as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Song Name", "Artist(s)", "Album"])

    # Write the song details to the CSV file
    for track in tracks:
        song_name = track["name"]
        artists = ", ".join([artist["name"] for artist in track["artists"]])
        album_name = track["album"]["name"]
        writer.writerow([song_name, artists, album_name])

# Check if there are more pages of results
while response_json["tracks"]["next"]:
    next_url = response_json["tracks"]["next"]

    # Send the next GET request to the Spotify API for the next page
    response = requests.get(next_url, headers=headers)

    # Parse the response JSON
    response_json = response.json()

    # Extract the relevant data from the response
    tracks = response_json["tracks"]["items"]

    # Append the song details to the CSV file
    with open("songs.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        for track in tracks:
            song_name = track["name"]
            artists = ", ".join([artist["name"] for artist in track["artists"]])
            album_name = track["album"]["name"]
            writer.writerow([song_name, artists, album_name])


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3A%27One+Direction%27&type=track&offset=0&limit=50', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4AK6F7OLvEQ5QYCBNiQWHq'}, 'href': 'https://api.spotify.com/v1/artists/4AK6F7OLvEQ5QYCBNiQWHq', 'id': '4AK6F7OLvEQ5QYCBNiQWHq', 'name': 'One Direction', 'type': 'artist', 'uri': 'spotify:artist:4AK6F7OLvEQ5QYCBNiQWHq'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'K

In [13]:
import pandas as pd
df = pd.read_csv('songs.csv')

df.head()

,Song Name,Artist,Album
0,What Makes You Beautiful,One Direction,Up All Night
1,Night Changes,One Direction,FOUR (Deluxe)
2,Story of My Life,One Direction,Midnight Memories (Deluxe)
3,Steal My Girl,One Direction,FOUR (Deluxe)
4,Drag Me Down,One Direction,Made In The A.M. (Deluxe Edition)


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Song Name  410 non-null    object
 1   Artist     410 non-null    object
 2   Album      410 non-null    object
dtypes: object(3)
memory usage: 9.7+ KB


In [15]:
df = df[df["Artist"] == "One Direction"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 266
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Song Name  143 non-null    object
 1   Artist     143 non-null    object
 2   Album      143 non-null    object
dtypes: object(3)
memory usage: 4.5+ KB


In [18]:
#drop duplicates on basis of song name
df = df.drop_duplicates(subset=['Song Name'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110 entries, 0 to 142
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Song Name  110 non-null    object
 1   Artist     110 non-null    object
 2   Album      110 non-null    object
dtypes: object(3)
memory usage: 3.4+ KB


In [19]:
df.to_csv('songs.csv', index=False)

In [27]:
def get_song_details(access_token, song_name, artist, album):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    params = {
        'q': f'track:"{song_name}" artist:"{artist}" album:"{album}"',
        'type': 'track',
        'limit': 1
    }
    response = requests.get(search_url, headers=headers, params=params)
    response_data = response.json()
    return response_data['tracks']['items'][0] if response_data['tracks']['items'] else None


In [30]:
input_file = 'songs.csv'
output_file = 'songs_with_details.csv'

# Open the input and output files
with open(input_file, 'r') as csv_file, open(output_file, 'w', newline='') as output_csv:
    reader = csv.DictReader(csv_file)
    fieldnames = reader.fieldnames + ['spotify_id', 'duration_ms', 'popularity']
    writer = csv.DictWriter(output_csv, fieldnames=fieldnames)
    writer.writeheader()

    # Get the access token

    # Iterate over each song in the input file
    for row in reader:
        song_name = row['Song Name']
        artist = row['Artist']
        album = row['Album']

        # Get the song details from Spotify API
        song_details = get_song_details(access_token, song_name, artist, album)

        if song_details:
            # Extract the relevant details
            spotify_id = song_details['id']
            duration_ms = song_details['duration_ms']
            popularity = song_details['popularity']

            # Write the updated row to the output file
            row['spotify_id'] = spotify_id
            row['duration_ms'] = duration_ms
            row['popularity'] = popularity
            writer.writerow(row)


In [31]:
def get_audio_features(access_token, track_id):
    audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    response = requests.get(audio_features_url, headers=headers)
    response_data = response.json()
    return response_data


In [33]:
input_file = 'songs_with_details.csv'
output_file = 'songs_with_audio_features.csv'

# Open the input and output files
with open(input_file, 'r') as csv_file, open(output_file, 'w', newline='') as output_csv:
    reader = csv.DictReader(csv_file)
    fieldnames = reader.fieldnames + ['acousticness', 'instrumentalness', 'liveness', 'loudness', 'danceability', 'energy', 'tempo', 'valence']
    writer = csv.DictWriter(output_csv, fieldnames=fieldnames)
    writer.writeheader()


    # Iterate over each song in the input file
    for row in reader:
        song_id = row['spotify_id']

        # Get the audio features from Spotify API
        audio_features = get_audio_features(access_token, song_id)

        # Extract the relevant audio features
        acousticness = audio_features['acousticness']
        instrumentalness = audio_features['instrumentalness']
        liveness = audio_features['liveness']
        loudness = audio_features['loudness']
        danceability = audio_features['danceability']
        energy = audio_features['energy']
        tempo = audio_features['tempo']
        valence = audio_features['valence']

        # Update the row with the audio features
        row['acousticness'] = acousticness
        row['instrumentalness'] = instrumentalness
        row['liveness'] = liveness
        row['loudness'] = loudness
        row['danceability'] = danceability
        row['energy'] = energy
        row['tempo'] = tempo
        row['valence'] = valence

        # Write the updated row to the output file
        writer.writerow(row)
